|||
|---|---|
|Title|nwtimetracking|
|Author|numbworks|
|Version|3.9.0|
||Please check [docs/docs-nwtimetracking.md](../docs/docs-nwtimetracking.md) before proceeding.|

#### Global Modules

In [1]:
from pandas import DataFrame
from typing import Optional

#### Local Modules

In [2]:
from nwtimetracking import SettingBag, DefaultPathProvider, YearProvider, SoftwareProjectNameProvider, TimeTrackingProcessor
from nwtimetracking import ComponentBag, TTMarkdownFactory
from nwpackageversions import LanguageChecker, RequirementChecker, RequirementSummary

#### Language/Requirement Checks

In [3]:
enable_language_status_check : bool = True
enable_requirement_status_check : bool = False

if enable_language_status_check:
    language_status : str = LanguageChecker().get_version_status(required = (3, 12, 5))
    print(language_status)

if enable_requirement_status_check:
    requirement_checker : RequirementChecker = RequirementChecker(logging_function = LambdaCollection().do_nothing_function())
    dockerfile_path : str = requirement_checker.get_default_devcointainer_dockerfile_path()
    requirement_status : Optional[RequirementSummary] = requirement_checker.try_check(file_path = dockerfile_path)
    print(requirement_status)

The installed Python version is matching the expected one (installed: '3.12.5', expected: '3.12.5').


#### Functions : Temp

#### Settings

In [4]:
setting_bag : SettingBag = SettingBag(
    excel_nrows = 1301,
    options_tt = ["display"],
    options_tts_by_month = ["display", "save"],
    options_tts_by_year = ["display"],
    options_tts_by_year_month = ["display"],
    options_tts_by_year_month_spnv = ["display"],
    options_tts_by_spn = ["display"],
    options_tts_by_spn_spv = ["display"],
    options_tts_by_hashtag = ["display"],
    options_tts_by_hashtag_year = ["display"],
    options_tts_by_efs = ["display"],
    options_tts_by_tr = ["display"],
    options_definitions = ["display"]
)


#### Main : Analysis

In [5]:
tt_processor : TimeTrackingProcessor = TimeTrackingProcessor(component_bag = ComponentBag(), setting_bag = setting_bag)
tt_processor.initialize()
tt_processor.process_tt()

Date,StartTime,EndTime,Effort,Hashtag,Descriptor,IsSoftwareProject,IsReleaseDay,Year,Month
2024-12-01,15:00,15:45,0h 45m,#python,nwpackageversions v1.8.0,True,True,2024,12
2024-12-01,15:45,16:15,0h 30m,#python,nwreadinglist v4.1.0,True,True,2024,12
2024-12-01,19:15,19:30,0h 15m,#python,nwreadinglist v4.1.0,True,True,2024,12
2024-12-01,19:30,20:15,0h 45m,#python,nwtimetracking v3.9.0,True,True,2024,12
2024-12-01,21:00,23:00,2h 00m,#python,nwtraderaanalytics v4.4.0,True,True,2024,12


In [6]:
tt_processor.process_tts_by_month()


Month,2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024
1,00h 00m,↑,18h 00m,↑,88h 30m,↓,80h 15m,↓,60h 00m,↓,29h 15m,↑,53h 00m,↓,00h 00m,↑,06h 00m,↑,45h 45m
2,00h 00m,↑,45h 30m,↑,65h 30m,↑,90h 45m,↓,73h 00m,↓,38h 00m,↓,31h 30m,↓,03h 00m,↑,24h 00m,↑,77h 45m
3,00h 00m,↑,20h 45m,↑,71h 45m,↑,89h 00m,↓,75h 30m,↓,35h 00m,↑,40h 30m,↓,06h 15m,↑,50h 15m,↑,77h 45m
4,00h 00m,↑,37h 30m,↑,68h 00m,↑,88h 30m,↓,59h 45m,↓,40h 45m,↓,19h 00m,↑,27h 30m,↓,19h 00m,↑,29h 30m
5,00h 00m,↑,53h 00m,↑,83h 00m,↑,91h 15m,↓,54h 45m,↓,14h 30m,↑,112h 45m,↓,49h 45m,↓,31h 00m,↑,43h 00m
6,00h 00m,↑,57h 45m,↓,37h 45m,↑,62h 00m,↓,29h 15m,↓,12h 00m,↑,54h 00m,↑,73h 30m,↓,24h 45m,↑,48h 00m
7,00h 00m,↑,46h 45m,↑,65h 30m,↑,69h 30m,↓,24h 15m,↑,34h 00m,↓,23h 30m,↑,51h 00m,↓,16h 30m,↑,67h 00m
8,00h 00m,↑,25h 45m,↑,45h 45m,↑,72h 00m,↓,06h 00m,↑,32h 00m,↑,110h 00m,↓,36h 30m,↑,41h 30m,↓,32h 45m
9,00h 00m,↑,89h 30m,↓,43h 45m,↑,64h 00m,↓,39h 00m,↑,44h 00m,↓,43h 30m,↑,69h 00m,↓,50h 15m,↓,48h 00m
10,08h 00m,↑,82h 15m,↓,64h 30m,↓,46h 45m,↓,45h 30m,↑,48h 00m,↓,35h 30m,↑,38h 30m,↓,20h 00m,↑,101h 30m


This content (id: 'tts_by_month') has been successfully saved as '/home/nwtimetracking/TIMETRACKINGBYMONTH.md'.
